# DiSSECT Playground

In [ ]:
# Import dependencies
from dissect.analysis.data_processing import load_curves, load_trait, find_outliers, flatten_trait
from dissect.analysis.visualization import violin, multiplot, normalized_barplot, normalized_bubbleplot

In [ ]:
# Define your data source or use the default
SOURCE = 'https://dissect.crocs.fi.muni.cz/static/data/'

In [ ]:
# Load curves
curves_source = load_curves(SOURCE)
curves_source

In [ ]:
# Filter curves
curves = curves_source.copy()
curves = curves[curves["bitlength"] == 192] # Filter curves with bitlength 224
curves = curves[curves["cofactor"] == 1] # Filter curves with cofactor 1
# curves = curves[curves["curve"].str.contains("x962")] # Filter curves simulated with x962 method
curves = curves[(curves["category"].isin(["secg","brainpool_sim_192"]))]
curves.reset_index(inplace=True, drop=True)
curves

In [ ]:
# Load trait results
trait_source = load_trait(SOURCE, "a06") # Select trait
trait_source

In [ ]:
# Filter trait results
trait = curves.merge(trait_source, "inner", on="curve", copy=True)
trait = trait[trait["deg"].isin([2,3])] # Select params
trait.reset_index(inplace=True, drop=True)
trait

In [ ]:
# prepare for barplot
trait2 = trait[trait["deg"]==2]
trait3 = trait[trait["deg"]==3]
graphs = multiplot(1,2,height=5,width=10)
feature = "least"
for results, graph in zip([trait2,trait3],graphs):
    std,sim = results[results["standard"]==True], results[results["standard"]==False]
    normalized_barplot(std[[feature]],sim[[feature]],graph)

In [ ]:
# prepare for bubbleplot
trait2 = trait[trait["deg"]==2]
trait3 = trait[trait["deg"]==3]
graphs = multiplot(1,2,height=5,width=10)
features = ("least","full")
for results, graph in zip([trait2,trait3],graphs):
    std,sim = results[results["standard"]==True], results[results["standard"]==False]
    normalized_bubbleplot(std[[*features]],sim[[*features]],graph)

In [ ]:
# prepare for bubbleplot
std = trait[trait["standard"]==True][["curve","deg","ratio_sqrt"]]
sim = trait[trait["standard"]==False][["curve","deg","ratio_sqrt"]]
std = flatten_trait("a06",std,[2,3]).drop(["curve"],1)
sim = flatten_trait("a06",sim,[2,3]).drop(["curve"],1)
normalized_bubbleplot(std,sim)

In [ ]:
violin(trait, "ratio_sqrt")

In [ ]:
find_outliers(trait, ["least", "full"])